# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

file = pd.read_csv("../output_data/cities.csv")
file.dropna()
file.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations
locations = file[["Lat", "Lng"]]
humidity = file["Humidity"].astype(float)

In [21]:
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations,
                                dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
temp_file=file.loc[(file['Max Temp']>=70)&(file['Max Temp']<=80)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']==0)]
ideal_weather.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -18.0445459,
                    "lng": 13.833306
                },
                "viewport": {
                    "northeast": {
                        "lat": -18.04335112010728,
                        "lng": 13.83455742989272
                    },
                    "southwest": {
                        "lat": -18.04605077989272,
                        "lng": 13.83185777010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Opuwo Country Lodge",
            "photos": [
                {
                    "height": 15

{
    "html_attributions": [],
    "next_page_token": "AcYSjRhIvlUeQDInjV6W2Trm8lnUOiYchBJZk8F10AHz3TnukaeDhmWi2QnLSZc08nV43UFV00ghDdTOrJkLipG8WqUbG3uTgU3FcV8dYUeqLBDKhv7uFX3uiLddhHrZOX7DvFu-e2IE6UsVfRArqyI9A5x9939Fd28Y8gQNZ2ugcGAndXHf0dGk9MdidY-aMku_9__RaC5fBR1WjGlzk4h5REzdXNZ1LrzZ_0kjgx5RdPQ5XfBs5DjPmPG9mSd7H_4L5E_JGjn7Sqkck88rM9JpmdZxV13BILaRLTbZ1j0ie7204ZQlxqjv0wvUcBQSwjdsDgouuZz0KHC7hMDpkdkBC1-s0ehIe2d8qfx3s2KBmGtgAuPl8uvii0YmEl_xMFt84Y7_CZzBSZlBPfuUYzWnJeE6X-9fOgnHf75QLs8sxTp88iyZs6tcrdgL_EXTxmcR",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.6441728,
                    "lng": -46.1058822
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.64295892010728,
                        "lng": -46.10472667010728
                    },
                    "southwest": {
                        "lat": -20.6456

{
    "html_attributions": [],
    "next_page_token": "AcYSjRgpFBGSLWhNgmHmE-NZxD8LcLQD0GBQG7fHZjkZu8778V45cq9KsTXWdE032hWeEW-Is0YCHNbtg1EHNvbvfdZFQ_XgsLrV8wgMO0upriKBXWAPvTFOLs2_waCbqWrEn2X3CnyXd29GkAsQL2jBc9rA1FYzcTev8AdyZE5aD5gi-ps_AGFsvM2ThsdJBWRovaBmc89EkY4n5lWedpzjOICZpYnFxPM_AS-d6_6NcHHJlBlNsNzvY2lC-gI0s7C1s_Yxhv1uh3c5PorEuIsKipKuhRvXM1asHPTcmGTuBph43CVS7kBddMQMD1FjvUBdh0eP5mBK16TKUY_3CjEh9iYtGqjJoz8WVGCr10vnNOfYODtC5n19oscGqYzICnSH5WBT4ul4zhHc1tLHVNRcvRDCZjFqwaP-fXLF4aXqewRtVPSIBBpxyxanvJlT6eK0",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -30.0375932,
                    "lng": -50.154824
                },
                "viewport": {
                    "northeast": {
                        "lat": -30.03635827010728,
                        "lng": -50.15343137010728
                    },
                    "southwest": {
                        "lat": -30.03905

{
    "html_attributions": [],
    "next_page_token": "AcYSjRgGlghnSCzSRlpx2sg825TtRvjlQTcNusMKIKfvje5HjKMmbJO4LZQm0l8SCX55EdRre3Qun4MsS8FsnlEa1OlEImAm51aqAd9dhR2COEB-2F__uR2l1DCNt_OX_8thFcpXDW6u2XrtifiSW0Y5_A2_U-aob0SFrxisv8dCdHWoTgb29AeLtCi150bZ0JP60rC6DVeklKu8CuQ_Mwa4iTb1sec7wSqEpav3ZGrEts3sTnzHQeYawbJ_aoli_edjfmFeECTwvpgIMJpCedNwltwGVNhFwypMmpgQSEafBimiyQFAUQAk1LE0OMx8Pr-dbrMJ2CYOcrN-X-VhFV5z_4AQqLzod2V_jmtbFGybPl5Rsb4eqZ-1bwfcjCmT3GpD5EA2gpnyUvu3tauxhRofpqAL9PdInVvQOjRxBWPeL08ANFbHU8WjlBNLF5flFHxw",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.7351511,
                    "lng": 44.00123
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.73380652010728,
                        "lng": 44.00258547989272
                    },
                    "southwest": {
                        "lat": -20.73650617

{
    "html_attributions": [],
    "next_page_token": "AcYSjRhELNUaChWnOlvy21f4dnRj6BvWzur0lAWqormV6I1l3FaDNtKk08GfSN8ql4ovWnEpi40IDscpiFKzSPQM91V0GZ1EEBDl9Il-8eG8p__muQz5X8BgfMuK2tvACmu5FBVAmCKMMQmJZLRpyy6maL8C3FyVRvUmsls3EVhGmrtRLBp-sqEWqlKfZJsuKGi-X7V49ZbJwmqNT1GJPkdIqTQfxRANKl0wPTYC6RixAItDZCNc7s2Fj6JeiJOA-pBynzU2SZUIUMZROkm3TqgWHXlRYea-WffCeA1wCtKNo67keD9UkiVWjqFHy69Yqzs6hRmGfBeELrW3U2iZvN5jQeeMRo8lIj2btTjSCm3zYxVqEewwHwxirl-k6d0pgrYToHGZATSNfI9PTth7jbsHajt73TOCCVyYLnRA48FJzAiAILyFC0dqBlVtHKV-qwaS",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 15.4681771,
                    "lng": 74.3026909
                },
                "viewport": {
                    "northeast": {
                        "lat": 15.46967287989272,
                        "lng": 74.30385317989273
                    },
                    "southwest": {
                        "lat": 15.466973220

{
    "html_attributions": [],
    "next_page_token": "AcYSjRjyeVNGZQD-Lq_oYhW2v6lxMEOiRDdvrT9_1wIuWJeFjc7oXpIGxKED7KRgY26IRgbb6ivAoC5vPjYvjZFMOI9zmqiwpgG3jhyJOt3n_x9vXqMQdPjopgqhDhSSFwMZrYR8B7fc1dJUjBqDFAZFMJHRLKWvUR9Dlg9mBG-vplwibBvivnp6thBKwza5-_A-znvx2Tsoo59ikgZ3k2AMhtn4lPI161IvMUv7SNz70bqdzKDz4E7MTrOSw7-6dDH9tJHvL-G_1d18xmZQwhCwsgzQJvF6jZwEuS9INLLrkxRgL6OX_Lc619XDfL5JRLWq2ad8UBh7CL8ByNNnba60uj9WSYjvePVVf86qotEypQgw5NLzwDg8u4DwdeTATeS7Sq0BRhlI6vV0x_ggZjcoX1TGaYDR0igqsqAGHImAvmJmskEq3_p_YzN_QMwJWA1S",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 35.178516,
                    "lng": -2.9219246
                },
                "viewport": {
                    "northeast": {
                        "lat": 35.18002257989272,
                        "lng": -2.920701170107277
                    },
                    "southwest": {
                        "lat": 35.177322920

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 22.9142468,
                    "lng": 96.4490175
                },
                "viewport": {
                    "northeast": {
                        "lat": 22.91553782989272,
                        "lng": 96.45037157989272
                    },
                    "southwest": {
                        "lat": 22.91283817010727,
                        "lng": 96.44767192010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Golden Butterfly Hotel",
            "photos": [
                {
                    "height": 2

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 32.8859972,
                    "lng": 59.2199585
                },
                "viewport": {
                    "northeast": {
                        "lat": 32.88738412989272,
                        "lng": 59.22126852989272
                    },
                    "southwest": {
                        "lat": 32.88468447010727,
                        "lng": 59.21856887010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Sepehr Hotel",
            "photos": [
                {
                    "height": 3264,
     

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,Opuwo Country Lodge
1,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Balneário do Lago Hotel
2,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Lebanon Praia Hotel
3,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,BELO BEACH
4,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Shangri-La Jungle Resort
5,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Hôtel Mercure Rif Nador
6,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,Golden Butterfly Hotel
7,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,Sepehr Hotel


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))